In [33]:
# Version 3.0.1
# Title: Flow Distribution in a Doublet EGS
# Authors: Pranay Asai (UoU) & Robert Podgorney (INL)
# Edited by: Pranay Asai
# Date: 06/26/2021
# Updates:
#     1. Minor Corrections

# NOTE: Friction Factors jupyter notebook must in the same folder as this code to run

import numpy as np  #Importing NumPy
import math
import matplotlib.pyplot as plt
import random
from ipynb.fs.defs.FrictionFactors import * #Imports all the functions
import platform

In [45]:
#      o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|
#                     |              |              |              |
#                     /              /              /              /
#                     \              \              \              /
#                     /              /              /              /
#                     |              |              |              |
#      o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|

#                 |----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----o
#                 |              |              |              |
#                 /              /              /              /
#                 \              \              \              \
#                 /              /              /              /
#                 |              |              |              |
#  o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|

## NOTE: Only run this cell at the beginning, you will have to run the code below with WellsOrientation = 1, 
## and then re-run the code below with WellsOrientation = 3.
## DO NOT re-run this code after initial initialization




## NOTE: The row number is the desired maximum number of perforations
## The column number is the length of the flowrates in this case 10 flowrates are tested
Pressuredrop = np.zeros((30,10))

In [46]:
#ORIENTATION OF WELLS
# If 1, The wells are Parallel
# If 2, The wells are Anti-Parallel
# If 3, The wells are Non-Parallel  # Input the Difference between first and last fracture
# NOTE: Change to 3 after first run but DO NOT re-run initialization step above!!!!
WellsOrientation=1

In [47]:
#For Fracture
NumberOfFractures=1
TotalWellLength=3227 #m, Length of the well
BasePermeability=1e-12 #m2, #Fracture base Permebility
FractureHeight=100 #m Fracture entrance length
FractureWidth=5.0 #m Fracture entrance pseudo width
BaseFractureLength=100 #m Height of fractures
Area_FractureEntrance=FractureHeight*FractureWidth #Area of fracture for the fluid to enter

#For Pipe
Diameter_InjectionWell=0.18#*(0.5/0.1524)   #m Diameter of the Injection Pipe
Diameter_ProductionWell=0.18#*(0.5/0.1524)  #m Diameter of the Production Pipe
Area_InjectionWell=math.pi*Diameter_InjectionWell**2*0.25 #meters sq.
Area_ProductionWell=math.pi*Diameter_InjectionWell**2*0.25 #meters sq.
DensityWater=1000 #kg/s
e=0.015/1000  #roughness in m
BaseInjectionWellSection=TotalWellLength/NumberOfFractures #m Length of Inj pipe section
BaseProductionWellSection=TotalWellLength/NumberOfFractures #m Length of Prod pipe section

#For Perforation
LengthOfPerfZone=1 #m, length og each perf zone
Cd=0.75 #Discharge Coefficient
NumberOfPerfPerMeter= [  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14, 15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,29,  30] # Number of perforations per meter
NumberOfPerfs=LengthOfPerfZone*NumberOfPerfPerMeter #Number of Perforations
BaseDiameter_InjectionPerforation=0.003175*3#*0.5 #m, Diameter of perforations
BaseDiameter_ProductionPerforation=0.003175*3#*0.5 #m, Diameter of perforations

In [48]:
#Initialization
Pressure_Injection=np.zeros(NumberOfFractures+1)
Pressure_Production=np.zeros(NumberOfFractures+1)
Pressure_Fracture=np.zeros(NumberOfFractures)
Flowrate_Fracture=np.zeros(NumberOfFractures)
FractureLength=np.zeros(NumberOfFractures)
InjectionWellSection=np.zeros(NumberOfFractures+1)
ProductionWellSection=np.zeros(NumberOfFractures+1)
Permeability_Fracture=np.zeros(NumberOfFractures)
Diameter_InjectionPerforation=np.zeros(NumberOfFractures)
Diameter_ProductionPerforation=np.zeros(NumberOfFractures)
WellAngle=0


In [49]:
#Flow Rate and Pressure
Flowrate_Initial=[5,10,15,20,25,30,35,40,45,50] #kg/s
Pressure_Injection[0]=3e7
#storage = np.zeros((20,6))

In [50]:
#CONTROL VARIABLES

#Turn on pressure drop in injection well
InjectionWellActivate=1 #if 1, the pressure drop in the well is on
#Turn on pressure drop in production well
ProductionWellActivate=1 #if 1, the pressure drop in the well is on
#Activate Perforation
ActivateInjectionPerforations=1  #If 0, perforation pressure drop is zero.
#activate production well perforation
#NOTE: Change this to 1 in order to activate exit perforations
ActivateProductionPerforations=0

#Variable Permeablity
# If 0, All fractures have same permeability
# If 1, The permeability values are assigned at random with respect to the base value
# If 2, Option to input custom permeability values.
VariablePermeability=0




if WellsOrientation==3:
    #d1=float(input("Enter the difference between the first and last fracture size: "))
    d1 = 100
    BaseFractureLength=BaseFractureLength-d1/2
    WellAngle=math.atan((d1/2)/((NumberOfFractures-1)*BaseInjectionWellSection)) #Angle of Injection well
    print(math.degrees(WellAngle)*2)
#for i in range(NumberOfFractures):
#    Flowrate_Fracture[i]=Flowrate_Initial/NumberOfFractures
#    FractureLength[i]=abs(BaseFractureLength+(NumberOfFractures-i-1)*math.tan(WellAngle)*BaseInjectionWellSection+(NumberOfFractures-i-1)*math.tan(WellAngle)*BaseProductionWellSection)
for i in range(NumberOfFractures+1):
    InjectionWellSection[i]=BaseInjectionWellSection/math.cos(WellAngle)
    ProductionWellSection[i]=BaseProductionWellSection/math.cos(WellAngle)

#Adaptive Perforation (Still in test phase)
# If 0, Adaptive Perforations are turned off
# If 1, Adaptive Perforations are turned on
# If 2, Custom input for Perforations
AdaptivePerf=0

In [51]:
#Calculate Fracture Permeability
#Function is defined as:
# var1*BasePermeability*(10^var2), where Var1 varies from 1 to 10 and Var2 varies from -1, 0 or 1, at random.
if VariablePermeability!=2:
    Permeability_Fracture=FracturePermeability(NumberOfFractures,BasePermeability,FractureWidth,FractureHeight,VariablePermeability)
else:
    for i in range(NumberOfFractures):
        #Define any custom function
        if i%2==0:
            Permeability_Fracture[i]=BasePermeability/2
        else:
            Permeability_Fracture[i]=BasePermeability

In [52]:
#Calculate Perforations
Diameter_InjectionPerforation=AdaptivePerforation(WellsOrientation,VariablePermeability,AdaptivePerf,NumberOfFractures,Diameter_InjectionPerforation,BaseDiameter_InjectionPerforation,Permeability_Fracture)
Diameter_ProductionPerforation=AdaptivePerforation(WellsOrientation,VariablePermeability,AdaptivePerf,NumberOfFractures,Diameter_ProductionPerforation,BaseDiameter_ProductionPerforation,Permeability_Fracture)
print(Diameter_InjectionPerforation)
print(Diameter_ProductionPerforation)
print(Area_FractureEntrance)

[0.009525]
[0.009525]
500.0


In [53]:
#Define a custom fucntion for Perforation diameters
if AdaptivePerf==2:
    for i in range(NumberOfFractures-1):
        #Define any custom function
        Diameter_InjectionPerforation[i]=BaseDiameter_InjectionPerforation
        Diameter_ProductionPerforation[i]=BaseDiameter_InjectionPerforation
    Diameter_InjectionPerforation[2] = 0.0076
    Diameter_ProductionPerforation[2] = 0.0076
print(Diameter_InjectionPerforation)
print(Diameter_ProductionPerforation)


[0.009525]
[0.009525]


In [54]:
counter = 0
counter2 = 0
while counter2 <= (len(Flowrate_Initial)-1):
    while counter <= (len(NumberOfPerfs)-1):

        Pressure_Injection=np.zeros(NumberOfFractures+1)
        Pressure_Production=np.zeros(NumberOfFractures+1)
        Pressure_Fracture=np.zeros(NumberOfFractures)
        Flowrate_Fracture=np.zeros(NumberOfFractures)
        Pressure_Injection[0]=3e7
        MaxIterations=1000 #number of iterations
        #mmax=10000 #max number of iterations
        Tolerance=1e-6 #Tolerance
        FactorTest = 1000 #NumberOfFractures*FractureHeight
        Flowrate_Update=np.zeros(NumberOfFractures)
        Flowrate_Frac1=np.zeros(MaxIterations)
        Flowrate_Frac2=np.zeros(MaxIterations)
        Flowrate_Frac3=np.zeros(MaxIterations)

        for i in range(NumberOfFractures):
            Flowrate_Fracture[i]=Flowrate_Initial[counter2]/NumberOfFractures
            FractureLength[i]=abs(BaseFractureLength+(NumberOfFractures-i-1)*math.tan(WellAngle)*BaseInjectionWellSection+(NumberOfFractures-i-1)*math.tan(WellAngle)*BaseProductionWellSection)
        for i in range(NumberOfFractures+1):
            InjectionWellSection[i]=BaseInjectionWellSection/math.cos(WellAngle)
            ProductionWellSection[i]=BaseProductionWellSection/math.cos(WellAngle)


        for j in range(MaxIterations):
            Flowrate_Cumulative=Flowrate_Initial[counter2]/FactorTest
            #pressure drop in injection well
            for i in range(NumberOfFractures):
                if InjectionWellActivate==0:
                    Pressure_Injection[i+1]=Pressure_Injection[i]
                else:            
                    Pressure_Injection[i+1]=Pressure_Injection[i]-Fhal(e,Diameter_InjectionWell,Rep(Flowrate_Cumulative,Diameter_InjectionWell))*2*InjectionWellSection[i+1]*DensityWater*(Flowrate_Cumulative/Area_InjectionWell)**2/Diameter_InjectionWell
                Flowrate_Cumulative=Flowrate_Cumulative-Flowrate_Fracture[i]/FactorTest

            #Pressure drop in Production well
            Flowrate_Cumulative=Flowrate_Fracture[NumberOfFractures-1]/FactorTest
            if WellsOrientation!=2:
                #pressure drops in last Fracture
                if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                    Pressure_Production[NumberOfFractures]=Pressure_Injection[NumberOfFractures]-Pfdarcy(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,FractureLength[NumberOfFractures-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures-1])-Pperf(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[NumberOfFractures-1])-Pperf(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[NumberOfFractures-1])
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                    Pressure_Production[NumberOfFractures]=Pressure_Injection[NumberOfFractures]-Pfdarcy(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,FractureLength[NumberOfFractures-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures-1])-Pperf(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[NumberOfFractures-1])
                elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                    Pressure_Production[NumberOfFractures]=Pressure_Injection[NumberOfFractures]-Pfdarcy(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,FractureLength[NumberOfFractures-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures-1])-Pperf(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[NumberOfFractures-1])
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations
                    Pressure_Production[NumberOfFractures]=Pressure_Injection[NumberOfFractures]-Pfdarcy(Flowrate_Fracture[NumberOfFractures-1]/FactorTest,FractureLength[NumberOfFractures-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures-1])


                for i in range(NumberOfFractures,0,-1):
                    #turning off production well pressure drop
                    if ProductionWellActivate==0:
                        Pressure_Production[i-1]=Pressure_Production[i]
                    else:
                        Pressure_Production[i-1]=Pressure_Production[i]-Fhal(e,Diameter_ProductionWell,Rep(Flowrate_Cumulative,Diameter_ProductionWell))*2*ProductionWellSection[i-1]*DensityWater*(Flowrate_Cumulative/Area_ProductionWell)**2/Diameter_ProductionWell
                    Flowrate_Cumulative=Flowrate_Cumulative+Flowrate_Fracture[i-1]/FactorTest
                #Flow Rate in all Fractures
                Flowrate_Reinitilized=0
                for i in range(NumberOfFractures):
                    Pressure_Fracture[i]=Pressure_Injection[i+1]-Pressure_Production[i+1]
                    if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                        Flowrate_Fracture[i]=Q_all_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[i],Diameter_ProductionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                        Flowrate_Fracture[i]=Q_Production_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                        Flowrate_Fracture[i]=Q_Injection_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations            
                        Flowrate_Fracture[i]=Q_No_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i])*FactorTest

                    Flowrate_Reinitilized=Flowrate_Reinitilized+Flowrate_Fracture[i]
            else:
                #pressure drops in First Fracture
                if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                    Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[0])
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                    Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[0])
                elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations    
                    Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[0])
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations    
                    Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])

                #Pressure drop in Production well
                Flowrate_ProductionReinitilized=0
                for i in range(NumberOfFractures):
                    Flowrate_ProductionReinitilized=Flowrate_ProductionReinitilized+Flowrate_Fracture[i]/FactorTest
                    if ProductionWellActivate==0:
                        Pressure_Production[i+1]=Pressure_Production[i]
                    else:
                        Pressure_Production[i+1]=Pressure_Production[i]-Fhal(e,Diameter_ProductionWell,Rep(Flowrate_ProductionReinitilized,Diameter_ProductionWell))*2*ProductionWellSection[i-1]*DensityWater*(Flowrate_ProductionReinitilized/Area_ProductionWell)**2/Diameter_ProductionWell

                #Flow Rate in all Fractures including perf
                Flowrate_Reinitilized=0
                for i in range(NumberOfFractures):
                    Pressure_Fracture[i]=Pressure_Injection[i+1]-Pressure_Production[i]
                    if Pressure_Fracture[i]>0:
                        if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                            Flowrate_Fracture[i]=Q_all_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[i],Diameter_ProductionPerforation[i])*FactorTest
                        elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                            Flowrate_Fracture[i]=Q_Production_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_ProductionPerforation[i])*FactorTest
                        elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                            Flowrate_Fracture[i]=Q_Injection_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs[counter],Diameter_InjectionPerforation[i])*FactorTest
                        elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations            
                            Flowrate_Fracture[i]=Q_No_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i])*FactorTest                

                    else:
                        Flowrate_Fracture[i]=0.01
                    Flowrate_Reinitilized=Flowrate_Reinitilized+Flowrate_Fracture[i]

            #Flowrate_Frac1[j]=Flowrate_Fracture[0]
            #Flowrate_Frac2[j]=Flowrate_Fracture[1]
            #Flowrate_Frac3[j]=Flowrate_Fracture[2]
            #Calculating flowrate for next iteration
            for i in range(NumberOfFractures):
                Flowrate_Fracture[i]=(Flowrate_Initial[counter2])*Flowrate_Fracture[i]/Flowrate_Reinitilized

            Flowrate_Update=Flowrate_Update+np.array(Flowrate_Fracture)
            if j>50:
                Flowrate_Fracture=Flowrate_Update/j
        #print(MaxIterations)

        distpct=np.empty(NumberOfFractures)
        for i in range(NumberOfFractures):
            distpct[i]=100*(Flowrate_Fracture[i]/sum(Flowrate_Fracture))
        #print(distpct)
        ppp = (145.038*(max(Pressure_Injection)-min(Pressure_Production))/1e6)
        
        if Flowrate_Initial[counter2] == 5:
            if WellsOrientation == 1:
                #storage5[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2)] = ppp
            else:
                #storage5[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial))] = ppp
        elif Flowrate_Initial[counter2] == 10:
            if WellsOrientation == 1:
                #storage10[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+1)] = ppp
            else:
                #storage10[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+1)] = ppp
        elif Flowrate_Initial[counter2] == 15:
            if WellsOrientation == 1:
                #storage15[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+2)] = ppp
            else:
                #storage15[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+2)] = ppp
        elif Flowrate_Initial[counter2] == 20:
            if WellsOrientation == 1:
                #storage20[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+3)] = ppp
            else:
                #storage20[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+3)] = ppp
        elif Flowrate_Initial[counter2] == 25:
            if WellsOrientation == 1:
                #storage25[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+4)] = ppp
            else:
                #storage25[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+4)] = ppp
        elif Flowrate_Initial[counter2] == 30:
            if WellsOrientation == 1:
                #storage30[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+5)] = ppp
            else:
                #storage30[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+5)] = ppp
        elif Flowrate_Initial[counter2] == 35:
            if WellsOrientation == 1:
                storage35[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+6)] = ppp
            else:
                #storage35[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+6)] = ppp
        elif Flowrate_Initial[counter2] == 40:
            if WellsOrientation == 1:
                #storage40[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+7)] = ppp
            else:
                #storage40[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+7)] = ppp
        elif Flowrate_Initial[counter2] == 45:
            if WellsOrientation == 1:
                #storage45[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+8)] = ppp
            else:
                #storage45[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+8)] = ppp
        elif Flowrate_Initial[counter2] == 50:
            if WellsOrientation == 1:
                #storage50[NumberOfPerfs[counter]-1,0:3] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)*2+9)] = ppp
            else:
                #storage50[NumberOfPerfs[counter]-1,3:6] = distpct
                Pressuredrop[counter,(len(Flowrate_Initial)*2-len(Flowrate_Initial)+9)] = ppp
        

        counter = counter + 1
    counter2 = counter2 + 1
    counter = 0
print(Pressuredrop)
#print(storage5)
#storage = [storage5,storage10,storage15,storage20,storage25,storage30,storage35,storage40,storage45,storage50]


[[  772.40254387  2826.07419101  6159.43157625 10771.85804114
  16662.99607292 23832.6086306  32280.52643616 42006.62264381
  53010.79890784 65292.97700228]
 [  296.83252939   923.79413309  1879.30144594  3162.73780948
   4773.74571094  6712.08810936  8977.5957267  11570.14171716
  14489.62773505 17735.97555439]
 [  208.76400819   571.52004829  1086.68475514  1753.64147028
   2572.03268095  3541.62134617  4662.23818791  5933.75636037
   7356.07751787  8929.12343441]
 [  177.94002577   448.22411861   809.26891336  1260.45775156
   1801.43312045  2431.95797905  3151.86304933  3961.0214855
   4859.33494185  5846.72519242]
 [  163.67292534   391.15571687   680.86500945  1032.18414461
   1444.75560959  1918.34236341  2452.77512805  3047.9270577
   3703.69980667  4420.01514898]
 [  155.92289547   360.15559741   611.11474066   908.18366676
   1251.00486295  1639.34128825  2073.02366463  2551.9251463
   3075.94738756  3645.01216242]
 [  151.2498719    341.46350312   569.0575285    833.41528958

In [44]:
print(Pressuredrop[:,0])

[117.65692152  64.79448275  54.99624673  51.56443571  49.9752965
  49.11182513  48.59108812  48.25307018  48.02130725  47.8555188
  47.73284872  47.63954494  47.56693077  47.50931246  47.46282817
  47.42478366  47.39325295  47.36682966  47.34446745  47.32537483
  47.30894414  47.29470258  47.28227787  47.27137362  47.26175148
  47.25321807  47.24561512  47.23881212  47.23270065  47.22719008]


In [16]:



np.savetxt("1Pressuredrop.csv", Pressuredrop, delimiter=",", header = ','.join(['5 kg/s Parallel','10 kg/s Parallel','15 kg/s Parallel','20 kg/s Parallel','25 kg/s Parallel','30 kg/s Parallel','35 kg/s Parallel','40 kg/s Parallel','45 kg/s Parallel','50 kg/s Parallel']))